### 1. Genel Tanımlamalar ve Başlangıç Değerleri

In [ ]:
static Random rastgele = new Random(); // Rastgele sayı üretmek için kullanılan nesne
static int nesilBoyutu = 5;           // Bir nesilde kaç birey olduğunu belirler
static int genUzunlugu = 5;           // Bireylerin gen uzunluğu (5 bit)
static int nesilSayisi = 20;          // Kaç nesil boyunca çalışacağını belirler
static double mutasyonOrani = 0.05;   // Mutasyon ihtimali %5

### 2. Ana Program Akışı (Main Metodu)

In [ ]:
static void Main()
{
    int[] bireyler = BaslangicPopulasyonuOlustur();

    for (int nesil = 0; nesil < nesilSayisi; nesil++)
    {
        int[] uygunlukDegerleri = bireyler.Select(UygunlukFonksiyonu).ToArray(); 

        int enIyiIndeks = Array.IndexOf(uygunlukDegerleri, uygunlukDegerleri.Max());
        Console.WriteLine($"Nesil {nesil + 1}: En iyi birey = {bireyler[enIyiIndeks]}, Değer = {uygunlukDegerleri[enIyiIndeks]}");

        int[] yeniBireyler = new int[nesilBoyutu]; 
        for (int i = 0; i < nesilBoyutu; i += 2) // Yeni bireyleri oluştur
        {
            int ebeveyn1 = EbeveynSec(bireyler, uygunlukDegerleri);
            int ebeveyn2 = EbeveynSec(bireyler, uygunlukDegerleri);

            (int cocuk1, int cocuk2) = Caprazlama(ebeveyn1, ebeveyn2);
            yeniBireyler[i] = MutasyonUygula(cocuk1);
            if (i + 1 < nesilBoyutu) yeniBireyler[i + 1] = MutasyonUygula(cocuk2);
        }

        bireyler = yeniBireyler; // Yeni nesli eski neslin yerine koy
    }

    int[] sonUygunlukDegerleri = bireyler.Select(UygunlukFonksiyonu).ToArray();
    int enIyiSonIndeks = Array.IndexOf(sonUygunlukDegerleri, sonUygunlukDegerleri.Max());
    Console.WriteLine($"\nEn iyi çözüm: {bireyler[enIyiSonIndeks]}, Maksimum f(x) = {sonUygunlukDegerleri[enIyiSonIndeks]}");

    Console.ReadKey();
}

In [ ]:
Ana akışın temel adımları:
	1.	Başlangıç popülasyonunu oluştur → BaslangicPopulasyonuOlustur() çağrılır.
	2.	Her nesil için döngü başlar (for (int nesil = 0; nesil < nesilSayisi; nesil++))
        •	Bireylerin uygunlukları hesaplanır (UygunlukFonksiyonu).
        •	En iyi birey ekrana yazdırılır.
        •	Yeni bireyler oluşturmak için:
            1.	Ebeveyn seçimi yapılır (EbeveynSec() kullanılır).
            2.	Çaprazlama yapılır (Caprazlama() ile iki ebeveynin genleri birleştirilir).
            3.	Mutasyon uygulanır (MutasyonUygula() ile bazı genler değiştirilir).
	    •	Yeni nesil, eski neslin yerine geçer.
	3.	En iyi çözüm ekrana yazdırılır.

### 3. Başlangıç Popülasyonu Oluşturma

#### Neden rastgele.Next(0, 31) kullanıldı?
5 bitlik bir genetik kod kullanıyoruz. Yani 00000 (0) ile 11111 (31) arasında değerler üretiyoruz.

In [ ]:
static int[] BaslangicPopulasyonuOlustur()
{
    return Enumerable.Range(0, nesilBoyutu)
                     .Select(_ => rastgele.Next(0, 31))
                     .ToArray();
}

### 4. Uygunluk Fonksiyonu

#### Amaç: Bireyin başarısını ölçer. Burada hedef, x² değerini maksimize etmek.
Daha karmaşık problemler için burası değiştirilebilir.

In [ ]:
static int UygunlukFonksiyonu(int x) => x * x;

### 5. Ebeveyn Seçimi (Rulet Çarkı Seçimi)

- 	Rulet Çarkı Seçimi kullanılmıştır.
-	Daha yüksek uygunluk değeri olan bireylerin seçilme olasılığı daha fazladır.
-	Rastgele bir değer üretilerek toplam uygunluk içindeki yerine göre ebeveyn seçilir.

In [ ]:
static int EbeveynSec(int[] bireyler, int[] uygunlukDegerleri)
{
    int toplamUygunluk = uygunlukDegerleri.Sum();
    int rastgeleDeger = rastgele.Next(0, toplamUygunluk);
    int toplam = 0;
    for (int i = 0; i < nesilBoyutu; i++)
    {
        toplam += uygunlukDegerleri[i];
        if (toplam > rastgeleDeger) return bireyler[i];
    }
    return bireyler[0];
}

### 6. Çaprazlama (Crossover)

- İki ebeveynin genleri belirli bir noktadan bölünüp değiştiriliyor.
- Bu, yeni bireylerin oluşmasını sağlar ve genetik çeşitliliği artırır.

In [ ]:
static (int, int) Caprazlama(int ebeveyn1, int ebeveyn2)
{
    int nokta = rastgele.Next(1, genUzunlugu - 1);

    string ebeveyn1Binary = Convert.ToString(ebeveyn1, 2).PadLeft(genUzunlugu, '0');
    string ebeveyn2Binary = Convert.ToString(ebeveyn2, 2).PadLeft(genUzunlugu, '0');

    string cocuk1Binary = ebeveyn1Binary.Substring(0, nokta) + ebeveyn2Binary.Substring(nokta);
    string cocuk2Binary = ebeveyn2Binary.Substring(0, nokta) + ebeveyn1Binary.Substring(nokta);

    int cocuk1 = Convert.ToInt32(cocuk1Binary, 2);
    int cocuk2 = Convert.ToInt32(cocuk2Binary, 2);

    return (cocuk1, cocuk2);
}

### 7. Mutasyon Uygulama

- Belirli bir olasılıkla genetik mutasyon uygular.
- Bit değiştirme işlemi yapılarak yeni bireylerin oluşmasına katkı sağlar.

In [ ]:
static int MutasyonUygula(int gen)
{
    double olasilik = rastgele.NextDouble();
    if (olasilik < mutasyonOrani)
    {
        string binaryString = Convert.ToString(gen, 2).PadLeft(genUzunlugu, '0');
        char[] bitDizisi = binaryString.ToCharArray();

        int bitKonumu = rastgele.Next(0, genUzunlugu);
        bitDizisi[bitDizisi.Length - 1 - bitKonumu] ^= (char)1;

        string yeniBinary = new string(bitDizisi);
        gen = Convert.ToInt32(yeniBinary, 2);
    }
    return gen;
}